## Deliverable 2: Create a Customer Travel Destinations Map
Use input statements to retrieve customer weather preferences, then use those preferences to identify potential travel destinations and nearby hotels. Then, show those destinations on a marker layer map with pop-up markers.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bartica,GY,6.4000,-58.6167,79.12,96,98,3.44,overcast clouds
1,1,Half Moon Bay,US,37.4636,-122.4286,78.58,71,0,17.27,clear sky
2,2,Langsa,ID,4.4683,97.9683,78.44,84,43,1.43,scattered clouds
3,3,Saskylakh,RU,71.9167,114.0833,54.07,92,100,5.26,light rain
4,4,Cape Town,ZA,-33.9258,18.4232,57.11,91,75,16.11,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bartica,GY,6.4000,-58.6167,79.12,96,98,3.44,overcast clouds
1,1,Half Moon Bay,US,37.4636,-122.4286,78.58,71,0,17.27,clear sky
2,2,Langsa,ID,4.4683,97.9683,78.44,84,43,1.43,scattered clouds
5,5,Victoria,HK,22.2855,114.1577,85.24,83,100,18.99,overcast clouds
6,6,Hilo,US,19.7297,-155.0900,88.29,82,75,9.22,broken clouds
9,9,Katherine,AU,-14.4667,132.2667,84.52,24,0,9.22,clear sky
11,11,Bathsheba,BB,13.2167,-59.5167,83.01,83,40,16.11,scattered clouds
18,18,Kapaa,US,22.0752,-159.3190,84.18,81,100,13.80,light rain
24,24,Lorengau,PG,-2.0226,147.2712,82.62,77,51,7.00,broken clouds
28,28,Wakkanai,JP,45.4094,141.6739,78.78,53,20,17.27,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                229
City                   229
Country                229
Lat                    229
Lng                    229
Max Temp               229
Humidity               229
Cloudiness             229
Wind Speed             229
Current Description    229
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bartica,GY,6.4000,-58.6167,79.12,96,98,3.44,overcast clouds
1,1,Half Moon Bay,US,37.4636,-122.4286,78.58,71,0,17.27,clear sky
2,2,Langsa,ID,4.4683,97.9683,78.44,84,43,1.43,scattered clouds
5,5,Victoria,HK,22.2855,114.1577,85.24,83,100,18.99,overcast clouds
6,6,Hilo,US,19.7297,-155.0900,88.29,82,75,9.22,broken clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Mulege,MX,26.8833,-111.9833,87.26,58,29,5.39,scattered clouds
693,693,Charleston,US,32.7766,-79.9309,83.10,84,0,9.22,clear sky
694,694,Ixtapa,MX,20.7000,-105.2000,78.15,89,100,1.05,overcast clouds
695,695,Gaur,NP,26.7667,85.2667,84.22,72,93,20.29,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bartica,GY,79.12,overcast clouds,6.4000,-58.6167,
1,Half Moon Bay,US,78.58,clear sky,37.4636,-122.4286,
2,Langsa,ID,78.44,scattered clouds,4.4683,97.9683,
5,Victoria,HK,85.24,overcast clouds,22.2855,114.1577,
6,Hilo,US,88.29,broken clouds,19.7297,-155.0900,
9,Katherine,AU,84.52,clear sky,-14.4667,132.2667,
11,Bathsheba,BB,83.01,scattered clouds,13.2167,-59.5167,
18,Kapaa,US,84.18,light rain,22.0752,-159.3190,
24,Lorengau,PG,82.62,broken clouds,-2.0226,147.2712,
28,Wakkanai,JP,78.78,few clouds,45.4094,141.6739,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Determine if there are any empty rows.
hotel_df.notnull().sum()

City                   229
Country                229
Max Temp               229
Current Description    229
Lat                    229
Lng                    229
Hotel Name             229
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bartica,GY,79.12,overcast clouds,6.4000,-58.6167,Sloth Island Nature Resort
1,Half Moon Bay,US,78.58,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
2,Langsa,ID,78.44,scattered clouds,4.4683,97.9683,OYO 3328 Hotel Mori
5,Victoria,HK,85.24,overcast clouds,22.2855,114.1577,Mini Central
6,Hilo,US,88.29,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
690,Mulege,MX,87.26,scattered clouds,26.8833,-111.9833,Casa Granada
693,Charleston,US,83.10,clear sky,32.7766,-79.9309,The Mills House Wyndham Grand Hotel
694,Ixtapa,MX,78.15,overcast clouds,20.7000,-105.2000,hotel la terminal
695,Gaur,NP,84.22,overcast clouds,26.7667,85.2667,Attorney Office


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))